<a href="https://colab.research.google.com/github/Apurwa127/Hyper_Parameter_Tuning/blob/main/HyperParameterTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
import pandas as pd
df = pd.read_csv('/content/realest.csv')
df

,Price,Bedroom,Space,Room,Lot,Tax,Bathroom,Garage,Condition
0,53,2,967,5,39,652,2,0,0
1,55,2,815,5,33,1000,1,2,1
2,56,3,900,5,35,897,2,1,0
3,58,3,1007,6,24,964,2,2,0
4,64,3,1100,7,50,1099,2,2,0
...,...,...,...,...,...,...,...,...,...
152,44,3,820,5,27,NaN,1,0,1
153,43,2,593,4,30,465,2,1,0
154,53,3,716,6,30,585,1,2,1
155,62,4,951,7,30,895,2,1,0


In [116]:
df.isna().sum()

,0
Price,1
Bedroom,1
Space,11
Room,1
Lot,11
Tax,10
Bathroom,1
Garage,1
Condition,1


In [117]:
columns = ['Price', 'Bedroom', 'Bathroom', 'Garage', 'Condition']

df = df.dropna(subset = columns)
df

,Price,Bedroom,Space,Room,Lot,Tax,Bathroom,Garage,Condition
0,53,2,967,5,39,652,2,0,0
1,55,2,815,5,33,1000,1,2,1
2,56,3,900,5,35,897,2,1,0
3,58,3,1007,6,24,964,2,2,0
4,64,3,1100,7,50,1099,2,2,0
...,...,...,...,...,...,...,...,...,...
151,42,3,585,5,50,713,1,0,0
152,44,3,820,5,27,NaN,1,0,1
153,43,2,593,4,30,465,2,1,0
154,53,3,716,6,30,585,1,2,1


In [118]:
df.isna().sum()

,0
Price,0
Bedroom,0
Space,10
Room,0
Lot,10
Tax,9
Bathroom,0
Garage,0
Condition,0


In [119]:
avg = ['Space', 'Lot', 'Tax']

for i in avg:
  mean_val = df[i].mean()
  df[i] = df[i].fillna(mean_val)
df.isna().sum()

/tmp/ipython-input-119-1482745354.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = df[i].fillna(mean_val)


,0
Price,0
Bedroom,0
Space,0
Room,0
Lot,0
Tax,0
Bathroom,0
Garage,0
Condition,0


In [120]:
X = df.drop(['Price'], axis = 'columns')
X

,Bedroom,Space,Room,Lot,Tax,Bathroom,Garage,Condition
0,2,967,5,39,652,2,0,0
1,2,815,5,33,1000,1,2,1
2,3,900,5,35,897,2,1,0
3,3,1007,6,24,964,2,2,0
4,3,1100,7,50,1099,2,2,0
...,...,...,...,...,...,...,...,...
151,3,585,5,50,713,1,0,0
152,3,820,5,27,912,1,0,1
153,2,593,4,30,465,2,1,0
154,3,716,6,30,585,1,2,1


In [121]:
Y = df['Price']
Y

,Price
0,53
1,55
2,56
3,58
4,64
...,...
151,42
152,44
153,43
154,53


In [122]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [123]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_test, y_test)

LinearRegression()

In [124]:
model.score(x_test, y_test)

0.753973765359132

In [125]:
model.predict([[2,	967,	5,	39,	652,	2,	0,	0]])

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([48.92936308])

In [128]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

kf = KFold(n_splits = 3)
cvs = cross_val_score(LinearRegression(), X, Y, cv = kf)
cvs


array([0.72085197, 0.67552297, 0.70295019])

In [137]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


def find_best_score(x, y):
  algorithm = {
      'lasso':{
          'model' : Lasso(),
          'params': {
            'alpha': [1, 2],
            'fit_intercept' : [True, False]
          }
      },
      'DecisionTreeClassifier':{
          'model': DecisionTreeClassifier(),
          'params': {
            'criterion' :['gini', 'entropy'],
            'splitter' : ['best', 'random']
          }
      },
      'RandomForestClassifier':{
          'model' : RandomForestClassifier(),
          'params':{
            'n_estimators' : [100],
            'criterion':['gini', 'entropy']
          }
      }
  }
  scores = []
  kf = KFold(n_splits = 3)
  for algo_name, param in algorithm.items():
    gs = GridSearchCV(param['model'], param['params'], cv = kf)
    gs.fit(X, Y)
    scores.append({
        'model' : algo_name,
        'best_score' : gs.best_score_,
        'best_param' : gs.best_params_
    })
  return pd.DataFrame(scores, columns =['model', 'best_score', 'best_param'])

find_best_score(X, Y)

,model,best_score,best_param
0,lasso,1,"{'alpha': 1, 'fit_intercept': True}"
1,DecisionTreeClassifier,0,"{'criterion': 'entropy', 'splitter': 'random'}"
2,RandomForestClassifier,0,"{'criterion': 'gini', 'n_estimators': 100}"


In [142]:
ls = Lasso()
ls.fit(x_train, y_train)
ls.predict([[2,	967,	5,	39,	652,	2,	0,	0]])

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(


array([53.82843155])